## Environment Setup

In [1]:
! pip install langchain_community langchain_mistralai langchainhub langchain tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.6 MB/s eta 0:00:00
  

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<your key goes here>'
os.environ['MISTRAL_API_KEY'] = '<your key goes here>'


In [3]:
questions = [ "What does Moby Dick say about humanity's struggle against nature?",
              "How does Moby Dick explore the theme of obsession through Ahab's quest?",
              "How is Captain Ahab portrayed as both a hero and a villain in Moby Dick?",
              # "What motivates Ishmael to join the Pequod, and how does he change throughout the novel?",
              # "What does the white whale symbolize in Moby Dick, and how does it relate to Ahab's obsession?",
              # "How does the novel Moby Dick use the ocean as a symbol of the unknown?",
              # "How does Melville’s narrative style in Moby Dick contribute to the sense of adventure and mystery?",
              # "How does Ishmael’s perspective shape the reader’s understanding of the story in Moby Dick?",
              # "How does Moby Dick reflect 19th-century views on fate and destiny?",
              # "What philosophical questions does Melville raise about human existence and purpose in Moby Dick?",
              # "How does Melville use imagery to depict the sea as both beautiful and terrifying in Moby Dick?",
              # "How does Moby Dick describe the vastness and danger of the open sea?",
              # "What moral dilemmas do the crew members face in Moby Dick?",
              # "How does Moby Dick present Ahab's pursuit of revenge as both justified and self-destructive?"
              ]

question = questions[0]

## Indexing

In [4]:
from langchain_community.document_loaders import WebBaseLoader
# there can be mulitple urls
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/2701/pg2701.txt")
books = loader.load()

### Splitting the text

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(books)
len(splits)

1412


### vectorize chunks

In [6]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=MistralAIEmbeddings())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:169: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


## Retrieval

### Init Retriever

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

### Sub-questions Prompt

In [8]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are a helpful assistant designed to generate multiple sub-questions related to an input question.
Your goal is to break down the main question into distinct sub-problems that can be answered individually.
Generate three related search queries based on: {question} and output each query on a new line."""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_mistralai import ChatMistralAI

generate_queries = (
    prompt_perspectives
    | ChatMistralAI(temperature=0)
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

### Output original questions and their detailized sub-questions

In [9]:
for question in questions:
    print("-------------")
    print(question)
    sub_problems = generate_queries.invoke({"question":question})
    for sub_problem in sub_problems:
        print("\t"+sub_problem)

-------------
What does Moby Dick say about humanity's struggle against nature?
	1. "Themes of Moby Dick related to human struggle with nature"
	2. "Analysis of Moby Dick's portrayal of man versus nature"
	3. "Interpretations of Moby Dick's commentary on the human battle against nature"
-------------
How does Moby Dick explore the theme of obsession through Ahab's quest?
	1. "Moby Dick: Analysis of Ahab's obsession in the novel"
	2. "The role of obsession in Moby Dick's plot with a focus on Ahab's quest"
	3. "How Herman Melville uses the character of Ahab to depict obsession in Moby Dick"
-------------
How is Captain Ahab portrayed as both a hero and a villain in Moby Dick?
	1. "Depiction of Captain Ahab as a hero in Moby Dick"
	2. "Analysis of Captain Ahab's villainous characteristics in Moby Dick"
	3. "Balancing of hero and villain traits in Captain Ahab's character in Moby Dick"


In [10]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve

retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

<ipython-input-10-73df1d0ee15a>:10: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


3

# Generation

In [11]:
from operator import itemgetter
from langchain_mistralai import ChatMistralAI
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatMistralAI(temperature=0)

final_rag_chain = (
    {"context": retrieval_chain,
     "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)

for question in questions:
    print("-------------")
    print(question)
    rephrased_questions = generate_queries.invoke({"question":question})
    for rephrased_question in rephrased_questions:
        print("\t"+rephrased_question)
    print("\nAnswer: ")
    print(final_rag_chain.invoke({"question":question}))

-------------
What does Moby Dick say about humanity's struggle against nature?
	1. "Themes of Moby Dick related to human struggle with nature"
	2. "Analysis of Moby Dick's portrayal of man versus nature"
	3. "Interpretations of Moby Dick's commentary on the human battle against nature"

Answer: 
The text suggests that humanity's struggle against nature, as exemplified by Ahab's hunt for the white whale Moby Dick, can be overwhelming and often leads to a need for temporary diversions. Ahab recognizes that his crew needs nearer things to think of during their long night watches and that they require temporary interests and employments to keep them healthily suspended for the final dash. This implies that humanity's struggle against nature can be too daunting and that it is essential to balance it with more manageable challenges. The text also highlights the capricious and unreliable nature of sailors, suggesting that people in general are susceptible to fickleness and may not maintain t